In [2]:
def pmi(verb, table):
    
    import math
    import statistics
    from time import time
    start = time()
    
    prepositions = dict()
    for index, _ in table[table.lemma == verb].iterrows():
        if table['gramm'].loc[index][2] not in 'mpgn':
            words = 0
            left = 1
            while table['link'].loc[index - left] != 'PUNC' and words < 5 and index - left >= 0:
                words += 1 
                current_word = table['lemma'].loc[index - left]
                if table['POS'].loc[index - left] == 'S':
                    prepositions[current_word] = prepositions.get(current_word, 0) + 1 
                left +=1

            right = 1
            words = 0
            while table['link'].loc[index + right] != 'PUNC' and words < 5 and index + right < len(table):
                words += 1
                current_word = table['lemma'].loc[index + right]
                if table['POS'].loc[index + right] == 'S':
                    prepositions[current_word] = prepositions.get(current_word, 0) + 1 
                right +=1
                
    prep_pmi = dict()
    words_number = len(table[(table['link'] != 'PUNC') & (table['head'] != 'PUNC')])
    verb_freq = len(table[table.lemma == verb]) / words_number
    margin = statistics.mean(prepositions.values())
    
    for prep in prepositions.keys():
        if prepositions[prep] < margin:
            continue
        prep_freq = len(table[table.lemma == prep]) / words_number
        together_freq = prepositions[prep] / words_number
    
        prep_pmi[prep] = math.log(together_freq / (prep_freq * verb_freq))
    
    print('PMI calculation time: {}'.format((time() - start) / 60))
    return sorted(prep_pmi, key=prep_pmi.get, reverse=True)[:5]

def get_case(tag):
    if tag[0] == 'S':
        return tag[3]
    elif tag[0] == 'N':
        return tag[4]
    elif tag[0] == 'P':
        return tag[5]
    elif tag[0] == 'M':
        return tag[-1]

def get_config(tags, tokens, pmi_preps):
    config = []
    case = ''
    for (tag, token) in zip(tags, tokens):
        if tag == '' or tag[0] in 'RAI' or tag[0] == 'V' and tag[2] in 'mpg':
            continue
        elif tag[0] == 'S' and token not in pmi_preps:
            case = get_case(tag)
        elif tag[0] == 'S' and token in pmi_preps:
            config.append(tag[0])
        elif tag[0] in 'V':
            config.append(tag[0])
            case = ''
        elif tag[0] in 'NPM':
            if tag[0] in 'NP' and get_case(tag) == case:
                continue
            if len(config) != 0 and config[-1][0] == 'P' and tag[0] == 'N' and config[-1][1] == get_case(tag):
                config[-1] = 'N' + get_case(tag)
            if len(config) != 0 and config[-1][0] == 'M' and tag[0] == 'N' and get_case(tag) in 'ga':
                config[-1] = 'N' + get_case(tag)
            elif (len(config) == 0 or config[-1] != (tag[0] + get_case(tag))):
                config.append(tag[0] + get_case(tag))
            case = ''
    result = []
    for i in range(0, len(config)):
        if config[i][0] == 'M':
            continue
        if not ((config[i - 1][0] == 'M' or config[i - 1][0] == 'N')  and config[i] == 'Ng') or i == 0:
            if config[i][0] in 'PN':
                config[i] = 'NP' + config[i][1]
            result.append(config[i])
        
    for tag in tags[::-1]:
        if tag and tag in ['что', 'как','чтобы']:
            result.append(tag)
        elif tag and tag not in ['что', 'как','чтобы']:
            break
            
#         if config[i] == 'C' and config[i-1][1] == config[i+1][1]:
#             result.remove(config[i])
#             result.remove(config[i+1])
    return result

examples = dict()

def extractor(verb, table, filename):
    
    import csv
    from time import time
    start = time()
    
    configs = []
    
    global examples
    examples.clear()
    possible_preps = pmi(verb, table)
    
    with open(filename, 'w', encoding='utf-8') as infotable:
        writer = csv.writer(infotable, quoting=csv.QUOTE_NONE, escapechar='\\', quotechar='', delimiter='\t', lineterminator='\n')
        sentence_id = 0
        
        for index, _ in table[table.lemma==verb].iterrows():
            if table['gramm'].loc[index][2] not in 'mpgn':
                tags  = [''] * 11
                tokens = [''] * 11
                tags[5] = table['gramm'].loc[index]
                tokens[5] = table['token'].loc[index]
                words = 0
                left = 1
                change_gen = False
                skip_sentence = False
                if table['lemma'].loc[index - 1] == 'не' and index >= 1:
                    left += 1
                    change_gen = True
                while table['link'].loc[index - left] != 'PUNC' and words < 5 and index - left >= 0:
                    words += 1
                    tags[5 - words] = table['gramm'].loc[index-left]
                    tokens[5 - words] = table['token'].loc[index-left]
                    
                    left += 1

                right = 1
                words = 0
                while table['link'].loc[index + right] != 'PUNC' and words < 5 and index + right < len(table):
                    words += 1
                    tags[5 + words] = table['gramm'].loc[index + right]
                    tokens[5 + words] = table['token'].loc[index + right]
                    if tags[5 + words][0] == 'N' and get_case(tags[5 + words]) == 'g' and change_gen:
                        change_gen = False
                        tags[5 + words] = tags[5 + words][:4] + 'a' + tags[5 + words][5:]
                    right +=1

                if table['gramm'].loc[index + right] == ',' and words < 5 and table['gramm'].loc[index + right + 1] in ['что', 'как','чтобы']:
                    tags[6 + words] = table['lemma'].loc[index + right + 1]
                    tokens[6 + words] = table['token'].loc[index + right + 1]
                
                current_config = get_config(tags, tokens, possible_preps)
                writer.writerow(tags)
                writer.writerow(tokens)
                writer.writerow(current_config)
                writer.writerow(' ')
                configs.append(current_config)
                config_str = ' '.join(current_config)
                if (config_str not in examples or len(examples[config_str]) < 5) and sentence_id != table.sent_id.loc[index]:
                    examples[config_str] = examples.get(config_str, []) + [table.sent_id.loc[index]]
                    sentence_id = table.sent_id.loc[index]
    
    print('Extraction time: {}'.format((time() - start) / 60))
    return configs

In [3]:
def count_nouns(verb, table):
    
    from time import time
    start = time()
    
    nouns = dict()
    for index, _ in table[table.lemma == verb].iterrows():
        if table['gramm'].loc[index][2] not in 'mpgn':
            words = 0
            left = 1
            while table['link'].loc[index - left] != 'PUNC' and words < 5 and index - left >= 0:
                words += 1 
                current_word = table['lemma'].loc[index - left]
                if table['POS'].loc[index - left] == 'N':
                    nouns[current_word] = nouns.get(current_word, 0) + 1 
                left +=1

            right = 1
            words = 0
            while table['link'].loc[index + right] != 'PUNC' and words < 5 and index + right < len(table):
                words += 1
                current_word = table['lemma'].loc[index + right]
                if table['POS'].loc[index + right] == 'N':
                    nouns[current_word] = nouns.get(current_word, 0) + 1 
                right +=1
                
    print('Calculation time: {}'.format((time() - start) / 60))
    return sorted(nouns, key=nouns.get, reverse=True)[:5]

def extractor_nouns(verb, table, filename):
    
    import csv
    from time import time
    start = time()
    
    configs = []
    
    possible_preps = pmi(verb, table)
    target_nouns = count_nouns(verb, table)
    
    with open(filename, 'w', encoding='utf-8') as infotable:
        writer = csv.writer(infotable, quoting=csv.QUOTE_NONE, escapechar='\\', quotechar='', delimiter='\t', lineterminator='\n')
        
        for index, _ in table[table.lemma==verb].iterrows():
            if table['gramm'].loc[index][2] not in 'mpgn':
                tags  = [''] * 11
                tokens = [''] * 11
                tags[5] = table['gramm'].loc[index]
                tokens[5] = table['token'].loc[index]
                words = 0
                left = 1
                have_noun = False
                while table['link'].loc[index - left] != 'PUNC' and words < 5 and index - left >= 0:
                    words += 1                          
                    tags[5 - words] = table['gramm'].loc[index-left]
                    tokens[5 - words] = table['token'].loc[index-left]
                    if table['lemma'].loc[index-left] in target_nouns:
                        have_noun = True
                    left += 1

                right = 1
                words = 0
                while table['link'].loc[index + right] != 'PUNC' and words < 5 and index + right < len(table):
                    words += 1 
                    tags[5 + words] = table['gramm'].loc[index + right]
                    tokens[5 + words] = table['token'].loc[index + right]  
                    if table['lemma'].loc[index+right] in target_nouns:
                        have_noun = True
                    right +=1

                if table['gramm'].loc[index + right] == ',' and words < 5 and table['lemma'].loc[index + right + 1] in ['что', 'как','чтобы']:
                    tags[6 + words] = table['lemma'].loc[index + right + 1]
                    tokens[6 + words] = table['token'].loc[index + right + 1]
                
                current_config = get_config(tags, tokens, possible_preps)
                if have_noun:
                    writer.writerow(tags)
                    writer.writerow(tokens)
                    writer.writerow(current_config)
                    writer.writerow(' ')
                    configs.append(current_config)
    
    print('Extraction time: {}'.format((time() - start) / 60))
    return configs

In [4]:
import pandas as pd
document = pd.read_csv('.\corpora\corpus-i-part1.txt', delimiter='\t', header = 0)

In [23]:
target_configs = extractor_nouns('идти', document, 'target_idti.csv')

PMI calculation time: 0.475482169787089
Calculation time: 0.2191342274347941
Extraction time: 1.0392104824384054


In [86]:
configs2 = extractor('вести', document, 'vesti.csv')

PMI calculation time: 0.26421731313069663
Extraction time: 0.36559864282608034


In [83]:
configs3 = extractor('идти', document, 'idti.csv')

PMI calculation time: 0.4044350544611613
Extraction time: 0.6420624136924744


In [18]:
configs = extractor('найти', document, 'najti.csv')

PMI calculation time: 0.2566649913787842
Extraction time: 0.3495704015096029


In [44]:
pmi('найти', document)

PMI calculation time: 0.2642288049062093


['для', 'в', 'у', 'из', 'на']

In [75]:
configs

[['E-', 'V', 'En'],
 ['En', 'Ea', 'V'],
 ['Eg', 'E-', 'V'],
 ['V', 'Ea'],
 ['V', 'Ea', 'Eg'],
 ['Eg', 'En', 'V', 'Ea', 'Eg'],
 ['En', 'Ea', 'S', 'El', 'V', 'S', 'El', 'Eg', 'El'],
 ['Eg', 'V'],
 ['En', 'V'],
 ['Eg', 'Ea', 'V'],
 ['En', 'Ea', 'V'],
 ['Ea', 'En', 'V', 'S', 'Eg'],
 ['En', 'E-', 'V'],
 ['En', 'E-', 'V'],
 ['S', 'Ea', 'E-', 'V'],
 ['Eg', 'V', 'Ea'],
 ['Ea', 'V', 'S', 'El'],
 ['V', 'E-', 'Ed', 'Ea'],
 ['En', 'V', 'Ea', 'S', 'Eg'],
 ['V', 'En', 'V', 'Ea', 'S', 'El', 'Eg'],
 ['En', 'Eg', 'S', 'El', 'V', 'Ea', 'S', 'El', 'E-', 'Eg'],
 ['V'],
 ['V'],
 ['En', 'V', 'En', 'E-'],
 ['V', 'Ea'],
 ['E-', 'En', 'Ed', 'Eg', 'V'],
 ['V', 'S', 'En'],
 ['En', 'Eg', 'S', 'Eg', 'V', 'S', 'En'],
 ['Ed', 'V', 'S', 'El', 'Ea', 'V'],
 ['En', 'V', 'Eg', 'S', 'Eg'],
 ['Ed', 'Ea', 'V', 'V', 'Ed', 'Ea'],
 ['En', 'V', 'S', 'El', 'Eg'],
 ['V', 'Ea', 'E-', 'Ea'],
 ['En', 'V', 'V', 'Ea'],
 ['V', 'Ea'],
 ['En', 'V', 'Ea', 'V', 'Eg'],
 ['Eg', 'En', 'Eg', 'V', 'S', 'El'],
 ['S', 'El', 'En', 'V', 'Ea', 'Eg']

In [20]:
def get_sent(table, sent_num):
    import string
    result = ' '.join(table[table.sent_id == sent_num].token)
    result = result.replace('\t', '\"')
    for punc in ['.', ',', '!', '?', ':']:
        result = result.replace(' ' + punc, punc)
    return result

In [5]:
pmi('вести', document)

PMI calculation time: 0.2812260071436564


['к', 'с', 'о', 'по', 'на']

In [13]:
def mirror(config):
    config = config.split()
    new_config = []
    for i in range(len(config)):
        if config[i] == 'S':
            continue
        elif i > 0 and config[i - 1] == 'S':
            new_config.append(config[i - 1] + ' ' + config[i])
        else:
            new_config.append(config[i])
    return ' '.join(new_config[::-1])

In [16]:
mirror('S Nl Na Nn V')

'V Nn Na S Nl'

In [40]:
import random, statistics
config_freq = {}

for i in configs:
    config_str = ' '.join(i)
    if mirror(config_str) < config_str and mirror(config_str).split() in configs:
        config_str = mirror(config_str)
    config_freq[config_str] = config_freq.get(config_str, 0) + 1

med = statistics.median(config_freq.values())
average = statistics.mean([i for i in config_freq.values() if i > med])

# start = 1
# for chain in config_freq:
    
for chain in sorted(config_freq, key=config_freq.get, reverse=True):
    print(chain, ':', config_freq[chain],'\n', get_sent(document, random.choice(examples[chain])),'\n')
    if config_freq[chain] < average:
        break

NPa V NPn : 70 
 Комвзвода не нашел ни машины, ни экипажа, а только воронку в три метра. 

NPa V : 65 
 На другой день после съемок поехал в гостиницу и, закрывая за собой дверь, сказал всем: " Прощайте! " А наутро его нашли мертвым. 

NPn V : 42 
 В Польше начались слушания об экстрадиции Игнатенко Убедительных доказательств против Шубина и Шамиряна следователи не нашли, и в конце октября этого года их выпустили под залог в 1, 5 миллиона рублей. 

V : 27 
 Но мы изъяли из оборота только то, что нашли. 

NPg V : 27 
 Если открыть список реферативных научных журналов, то, к примеру, исследований влияния Луны практически не найдешь. 

NPg V NPn : 24 
 Пожалуй, самое образное сравнение непростой проблемы разумного использования солидных финансов нашел губернатор Дмитрий Мезенцев, встретившийся в день открытия выставки с главврачами всех лечебно-профилактических учреждений Иркутской области. 

NPn V NPa S NPl : 17 
 Эта логика нашла отражение в данных об акционерах банков. 

NPn NPa V : 15